In [74]:
#Defining the webscrapper class
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

class Scrap:
    def __init__(self, lei, n=None, inciso=None, alinea=None):
        """Initializing the driver"""
        self.lei = lei
        self.n = n
        self.inciso = inciso
        self.alinea = alinea
        
        self.m = 0
        self.a = 0
        self.an = 0
        self.at = 0
        self.inc = 0
        self.art = str('Art. ' + str(self.n))
        self.lista_paragrafo = []
        self.lista_inciso = []
        self.lista_alinea = []
        
        url = 'https://www2.camara.leg.br/busca/?q=' + lei
        option = Options()
        option.headless = True
        driver = webdriver.Firefox(options=option)
        driver.get(url)
        driver.find_element_by_xpath("//div[@id='resultadoBusca']//ul//li//span//a").click()
        driver.find_element_by_xpath("//div[@class='sessao']//a").click()
        element = driver.find_element_by_xpath("//div[@class='texto']")
        self.titulo = driver.find_element_by_class_name('ementa')
        html_content = element.get_attribute('outerHTML')
        self.titulo = self.titulo.get_attribute('outerHTML')
        #time.sleep(2)
        driver.quit()

        self.soup = BeautifulSoup(html_content, 'lxml')
        self.texto = self.soup.find_all(text=True)

        self.titulo = BeautifulSoup(self.titulo, 'lxml')
        self.titulo = self.titulo.find_all(text=True)
        
    def title(self):
        """Returns the law title""" 
        for i in range(len(self.titulo)):
            self.titulo[i] = self.titulo[i].replace('\n', '')
            self.titulo[i] = self.titulo[i].replace('\t', '')
        for i in range(len(self.titulo)):
            if '' in self.titulo:
                self.titulo.remove('')
            return(self.titulo[0])
    
    def paragraph(self):
        """Returns every laws paragraph"""
        for i in range(len(self.texto)):
            self.texto[i] = self.texto[i].replace('\n', '')
            self.texto[i] = self.texto[i].replace('\t', '')
            self.texto[i] = self.texto[i].replace('\xa0', '')
        for i in range(len(self.texto)):
            if '' in self.texto:
                self.texto.remove('')

        for i in range(len(self.texto)):
            #caso do inciso vir primeiro
            if self.inciso == self.texto[i].split() and self.inc == 0:
                for j in range(len(self.texto)):
                    self.lista_inciso.append(self.texto[j])
                    self.inc += 1

            if '§' in self.texto[i]:
                self.lista_paragrafo.append(self.texto[i])

                
            if self.a != 0 and self.an == 0 and self.alinea != '' and self.alinea in self.texto[i]:
                self.lista_alinea.append('Alinea '+ self.alinea + self.texto[i+1])
                self.an += 1
                

            if 'CAPÍTULO' in self.texto[i]:
                break

            if self.art in self.texto[i]:
                self.lista_paragrafo.append(self.texto[i])
                self.m += 1
            if self.m != 0 and '§' in self.texto[i]:
                self.lista_paragrafo.append(self.texto[i])
        return(self.lista_paragrafo)

In [78]:
teste = Scrap(lei='lei 1.060', n=4)

In [79]:
teste.title()

'Estabelece normas para a concessão de assistência judiciária aos necessitados. '

In [80]:
teste.paragraph()

['Art. 4ºA parte, que pretender gozar os benefícios da assistência judiciária, requererá ao Juiz competente lhos conceda, mencionando, na petição, o rendimento ou vencimento que percebe e os encargos próprios e os da família. ',
 ' § 1º A petição será instruída por um atestado de que conste ser o requerente necessitado, não podendo pagar as despesas do processo. Êste documento será expedido, isento de selos e emolumentos, pela autoridade policial ou pelo Prefeito municipal. ',
 ' § 1º A petição será instruída por um atestado de que conste ser o requerente necessitado, não podendo pagar as despesas do processo. Êste documento será expedido, isento de selos e emolumentos, pela autoridade policial ou pelo Prefeito municipal. ',
 ' § 2º Nas capitais dos Estados e no Distrito Federal, o atestado da competência do Prefeito poderá ser expedido por autoridade expressamente designada pelo mesmo. ',
 ' § 2º Nas capitais dos Estados e no Distrito Federal, o atestado da competência do Prefeito pod